In [16]:
using Agents, Random, Distributions, UnicodePlots

Random.seed!(123);

┌ Info: Precompiling UnicodePlots [b8865327-cd53-5732-bb35-84acbb429228]
└ @ Base loading.jl:1273


In [17]:
mutable struct Human <: AbstractAgent
           id :: Int 
       status :: Symbol # :susceptible, :infected, or :removed
    infection :: Float64
end

agent_properties = [:status, :infection]

2-element Array{Symbol,1}:
 :status   
 :infection

In [18]:
function SIRModel(;
                              humans = 1000,         # Number of susceptible
                            infected = 10,           # Number of infected
                        contact_rate = 0.1,
                      infection_rate = 0.1,
                   initial_infection = Uniform(0, 1), # Probability distribution for time-since-infected
                           time_step = 1.0           # hours
                  )   

    model_properties = Dict(
                            :contact_rate   => contact_rate,
                            :infection_rate => infection_rate,
                            :total          => humans,
                            :infected       => infected,
                            :p_infection    => 0.0,
                            :time_step      => time_step,
                           )   

    model = ABM(Human, scheduler=fastest, properties=model_properties)

    # Initialize susceptible population
    susceptible = humans - infected

    for i = 1:susceptible
        add_agent!(model, :susceptible, 0.0)
    end 

    # Initialize infected population
    for i = 1:infected
        add_agent!(model, :infected, 0) #rand(initial_infection))
    end 

    return model
end

model = SIRModel() # test whether function works...

AgentBasedModel with 1000 agents of type Human
 no space
 scheduler: fastest
 properties: Dict{Symbol,Real}(:infection_rate => 0.1,:p_infection => 0.0,:contact_rate => 0.1,:infected => 10,:total => 1000,:time_step => 1.0)

In [19]:
function agent_step!(agent, m)
    
    if agent.status == :removed
        return nothing
        
    elseif agent.status  == :susceptible
        
        # Define a stochastic process that determines whether the 
        # agent becomes infected.
        
        # The (random) number of infected people this agent contacts
        contacted = round(Int, rand(Gamma(m.infected, m.contact_rate * m.time_step)))
        
        # The probability that a susceptible is infected...
        p_infected = 1 - cdf(Binomial(contacted, m.infection_rate), 1)
        
        if rand(Bernoulli(p_infected))
            agent.status = :infected
        end
        
    elseif agent.status == :infected
        
        agent.infection += m.time_step
        
        # agent may become removed
    end 

    return nothing
end

agent_step! (generic function with 1 method)

In [21]:
model = SIRModel()
data = step!(model, agent_step!, 10, agent_properties)

infected = filter(x -> x.status === :infected, data)

,id,status,infection,step
,Int64,Symbol,Float64,Int64
1,991,infected,0.0,0
2,992,infected,0.0,0
3,993,infected,0.0,0
4,994,infected,0.0,0
5,995,infected,0.0,0
6,996,infected,0.0,0
7,997,infected,0.0,0
8,998,infected,0.0,0
9,999,infected,0.0,0


In [22]:
infection = infected[!, :infection]
UnicodePlots.histogram(infection)

                ┌                                        ┐ 
   [ 0.0,  2.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 38   
   [ 2.0,  4.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 38   
   [ 4.0,  6.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 38   
   [ 6.0,  8.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 32         
   [ 8.0, 10.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 24                
   [10.0, 12.0) ┤▇▇▇▇▇▇▇▇▇ 10                              
                └                                        ┘ 
                                Frequency